In [2]:
import pandas as pd
import numpy as np
import os,sys
os.chdir('C:/Users/arpit.goel/Documents/Projects/Kaggle/25_KingsOfML')
import matplotlib.pyplot as plt
%matplotlib inline

In [30]:
# df_train=pd.read_csv('01.RawData/train.csv')
# df_test=pd.read_csv('01.RawData/test_BDIfz5B.csv')
# df_cmpgn=pd.read_csv('01.RawData/campaign_data.csv')
# df_sample=pd.read_csv('01.RawData/sample_submission_4fcZwvQ.csv')
# df_train['send_date']=pd.to_datetime(df_train['send_date'],format='%d-%m-%Y %H:%M')
# df_test['send_date']=pd.to_datetime(df_test['send_date'],format='%d-%m-%Y %H:%M')
# df_train.sort_values(by=['user_id','send_date'],inplace=True)
# df_test.sort_values(by=['user_id','send_date'],inplace=True)
# store=pd.HDFStore('01.RawData/DataStore.h5')
# store['train']=df_train
# store['test']=df_test
# store['cmpgn']=df_cmpgn
# store['sample']=df_sample
# store.close()

In [31]:
df_train=pd.read_hdf('01.RawData/DataStore.h5', 'train')
df_test=pd.read_hdf('01.RawData/DataStore.h5', 'test')
df_cmpgn=pd.read_hdf('01.RawData/DataStore.h5', 'cmpgn')
df_sample=pd.read_hdf('01.RawData/DataStore.h5', 'sample')

In [27]:
print df_train.shape,df_test.shape

(1023191, 6) (773858, 4)


In [33]:
#Get unique number of user ID's and campaign ID's in train and test
a1=df_train['user_id'].drop_duplicates().shape[0]
a2=df_test['user_id'].drop_duplicates().shape[0]
a3=df_train['campaign_id'].drop_duplicates().shape[0]
a4=df_test['campaign_id'].drop_duplicates().shape[0]
a5=a1+a2-pd.concat([df_train['user_id'],df_test['user_id']]).drop_duplicates().shape[0]
a6=a3+a4-pd.concat([df_train['campaign_id'],df_test['campaign_id']]).drop_duplicates().shape[0]
print a1,a2,a5
print a3,a4,a6

168236 198219 145737
26 26 0


In [7]:
df_train['source']='train'
df_test['source']='test'
master=pd.concat([df_train,df_test])
master['date']=master['send_date'].dt.date
master['count']=1

In [58]:
master.groupby(['date','campaign_id','source'])[['count','is_open','is_click']].sum().to_clipboard()


In [62]:
# df_train.groupby(['user_id'])['is_click'].mean()
%run 00.Scripts/01.DefaultOpenClickRate.py

In [35]:
df_train['campaign_rank']=df_train.groupby('user_id')['send_date'].rank()


In [68]:
df_train['count']=1
df_train.groupby(['campaign_rank'])[['count','is_open','is_click']].sum().to_clipboard()


In [73]:
user_details=df_train.groupby(['user_id'])[['is_open','is_click']].sum().add_prefix('user_')
user_details=np.clip(user_details,0,1)
train1=pd.merge(df_train,user_details,left_on='user_id',right_index=True)

In [85]:
get_oc_rates=lambda x: pd.Series({'open_rate':1.0*x['is_open'].sum()/x['count'].sum(),'click_rate':1.0*x['is_click'].sum()/x['count'].sum(),'click_per_open':1.0*x['is_click'].sum()/x['is_open'].sum()})
cmpgn_sigs0=train1.groupby('campaign_id').apply(get_oc_rates)
cmpgn_sigs1=train1[train1['user_is_open']==1].groupby('campaign_id').apply(get_oc_rates).add_prefix('open_user_')
cmpgn_sigs2=train1[train1['user_is_click']==1].groupby('campaign_id').apply(get_oc_rates).add_prefix('click_user_')
cmpgn_sigs=pd.concat([cmpgn_sigs0,cmpgn_sigs1,cmpgn_sigs2],axis=1)


In [109]:
cmpgn_summary=pd.merge(df_cmpgn,cmpgn_sigs,left_on='campaign_id',right_index=True)
cmpgn_summary=pd.concat([cmpgn_summary,pd.get_dummies(cmpgn_summary['communication_type'])],axis=1)
cmpgn_summary.corr().to_clipboard()


In [106]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(binary=True)
vocab_train=cv.fit(df_cmpgn[df_cmpgn['campaign_id']<=54]['subject']).vocabulary_.keys()
vocab_test=cv.fit(df_cmpgn[df_cmpgn['campaign_id']>54]['subject']).vocabulary_.keys()
vocab=list(set(vocab_train).intersection(set(vocab_test)))
print len(vocab_train),len(vocab_test),len(vocab)
print vocab

# word_freq=pd.DataFrame(cv.transform(cmpgn_summary['subject']).todense(),columns=sorted(cv.vocabulary_.keys())).sum()
# cv.transform(cmpgn_summary['subject']).todense()
# word_freq.sort_values(ascending=False)

140 135 37
[u'and', u'win', u'is', u'in', u'learning', u'av', u'go', u'newsletter', u'your', u'now', u'hackathons', u'for', u'data', u'opportunities', u'machine', u'to', u'much', u'2018', u'new', u'you', u'worth', u'more', u'datahack', u'business', u'000', u'prizes', u'hackathon', u'coming', u'exciting', u'with', u'on', u'ahead', u'science', u'register', u'us', u'chief', u'the']


In [5]:
df_test['send_date'].dt.month.value_counts()

2    339237
3    230228
1    204393
Name: send_date, dtype: int64

In [80]:
get_oc_rates=lambda x: pd.Series({'open_rate':1.0*x['is_open'].sum()/x['count'].sum(),'click_rate':1.0*x['is_click'].sum()/x['count'].sum()})
train1.groupby('campaign_id').apply(get_oc_rates)

,click_rate,open_rate
campaign_id,,
29,0.016549,0.065009
30,0.012615,0.141909
31,0.024436,0.171992
32,0.010600,0.148440
33,0.011449,0.203888
34,0.009807,0.047420
35,0.043921,0.170590
36,0.004767,0.131513
37,0.006482,0.122238


In [15]:
# master_1=pd.merge(master,df_cmpgn[['campaign_id','communication_type']],on='campaign_id')
t1=master_1.groupby(['user_id','communication_type'])['count'].sum().unstack()
np.clip(t1.reset_index().fillna(0),0,1).groupby(t1.columns.tolist())['user_id'].count().to_clipboard()

In [36]:
# master['campaign_rank']=master[['user_id','send_date']].groupby('user_id')['send_date'].rank()
# pd.crosstab(df_train['campaign_id'],df_train['campaign_rank']).to_clipboard()\
df_train['count']=1
df_train.groupby(['campaign_id','campaign_rank'])[['count','is_open','is_click']].sum().to_clipboard()

In [47]:
%%time
# group=df_train[df_train['user_id']==42199].sort_values(by='send_date')
open_rate_data=[]
for name,group in df_train.head(100).groupby('user_id'):
    group['cnt_rcv']=np.arange(len(group))
    #group['cnt_open']=group['is_open'].cumsum()-group['is_open']
    #group['cnt_click']=group['is_click'].cumsum()-group['is_click']
    open_rate_data.append(group)
open_rate_data=pd.concat(open_rate_data)

C:\Users\arpit.goel\AppData\Local\Continuum2\Anaconda2\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


Wall time: 905 ms


In [ ]:
valid_X['is_click'].shape

In [ ]:
valid_X['is_click'].describe()

In [ ]:
df_train.head()